<a href="https://colab.research.google.com/github/YoshiyukiKono/pulsar_for_beginners/blob/main/pulsar_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pulsar クライアント開発演習

## 概要

### 事前準備
 - 環境準備
  - Astraアカウント登録・サインイン
  - Astra DB データベース、キースペース作成
  - Astra Streaming テナント作成

 - 開発準備
  - Astra DB テーブル作成
  - Astra Streaming ネームスペース、トピック、サブスクリプション作成
  - Astra DB トークン生成
  - Astra Streaming トークン生成
 - データベース連携
  - Astra Streaming Sink作成

### 演習概要

#### 第一部 Pulsar データ入力
 - Pulsarクライアントライブラリインストール
 - Pulsar接続設定
 - Pulsarプロデューサー実行
  - Astra DBへのSink結果の確認
 - Pulsarコンシューマー実行
 - Astra DB CDC

#### 第二部 Astra DB CDC + Pulsarデータ出力
 - Astraコントロールプレーンでの操作
  - Astra DB CDC作成
  - Astra Streaming CDC用ネームスペース、トピック確認
  - Astra Streaming CDC用サブスクリプション作成
 - 本演習ノートブックでの操作
  - Pulsarコンシューマー実行

---

## 演習: 第一部 Pulsar データ入力

### Pulsarクライアントライブラリインストール

In [1]:
!pip install pulsar-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.6 MB/s eta 0:00:00


### Astra Streaming 接続設定

In [2]:
service_url = input("service url:")


service url:pulsar+ssl://pulsar-gcp-australiase1.streaming.datastax.com:6651


In [3]:
import getpass

token = getpass.getpass()

··········


In [6]:
print("token:" + token[0:5] + "..." + token[-5:])
print("token length:" + str(len(token)))

token: eyJhb...euW8Q
token length:548


In [7]:
your_tenant_name = input("tenant name:")

tenant name:cdc
namespace:default


In [17]:
your_namespace = input("namespace:")
your_topic_name = input("topic name:")
topic_url = 'persistent://' + your_tenant_name + '/' + your_namespace + '/' + your_topic_name
print("topic url:" + topic_url)

namespace:default
topic name:astrasink
topic url:persistent://cdc/default/astrasink


In [11]:
import pulsar

client = pulsar.Client(service_url,
                        authentication=pulsar.AuthenticationToken(token))
topic_partitions = client.get_topic_partitions(topic_url)
print(topic_partitions)

['persistent://cdc/default/astrasink']


## Pulsarプロデューサー実行

In [12]:
from pulsar.schema import *

class Message(Record):
    message_id = String()
    message = String()

producer = client.create_producer(topic_url,schema=JsonSchema(Message))

message = Message()

for i in range(10):
    message.message_id = str(i)
    message.message = "Hello" + str(i)
    producer.send(message)

producer.close()

## Pulsarコンシューマー実行

In [13]:
subscription_name = input("subsuctiption name:")

subsuctiption name:sub


In [14]:
import pulsar
import time

MODE_RUNNING = False

consumer = client.subscribe(topic_url, subscription_name)

try:
    waitingForMsg = True
    while waitingForMsg:
        try:
            msg = consumer.receive(2000)

            try:
                print("Received message '{}' id='{}'".format(msg.data(), msg.message_id()))
                # Acknowledging the message to remove from message backlog
                consumer.acknowledge(msg)
            except:
                consumer.negative_acknowledge(msg)

        except Exception as ex:
            print(ex)
            if MODE_RUNNING:
                print("Still waiting for a message...");
            else:
                print("Stop waiting for a message.");
                waitingForMsg = False
        time.sleep(1)

finally:
  consumer.close()

Received message 'b'{\n "message_id": "0",\n "message": "Hello0"\n}'' id='(435390,0,-1,-1)'
Received message 'b'{\n "message_id": "1",\n "message": "Hello1"\n}'' id='(435390,1,-1,-1)'
Received message 'b'{\n "message_id": "2",\n "message": "Hello2"\n}'' id='(435390,2,-1,-1)'
Received message 'b'{\n "message_id": "3",\n "message": "Hello3"\n}'' id='(435390,3,-1,-1)'
Received message 'b'{\n "message_id": "4",\n "message": "Hello4"\n}'' id='(435390,4,-1,-1)'
Received message 'b'{\n "message_id": "5",\n "message": "Hello5"\n}'' id='(435390,5,-1,-1)'
Received message 'b'{\n "message_id": "6",\n "message": "Hello6"\n}'' id='(435390,6,-1,-1)'
Received message 'b'{\n "message_id": "7",\n "message": "Hello7"\n}'' id='(435390,7,-1,-1)'
Received message 'b'{\n "message_id": "8",\n "message": "Hello8"\n}'' id='(435390,8,-1,-1)'
Received message 'b'{\n "message_id": "9",\n "message": "Hello9"\n}'' id='(435390,9,-1,-1)'
Pulsar error: TimeOut
Stop waiting for a message.


In [15]:
client.close()

---

## 演習 第二部 Astra DB CDC + Pulsarデータ出力

In [18]:
your_namespace = input("namespace:")
your_topic_name = input("topic name:")
topic_url = 'persistent://' + your_tenant_name + '/' + your_namespace + '/' + your_topic_name
print("topic url:" + topic_url)

namespace:astracdc
topic name:data-b04d69bb-6851-4bac-9703-8108dc560c41-cdc.cdc_test
topic url:persistent://cdc/astracdc/data-b04d69bb-6851-4bac-9703-8108dc560c41-cdc.cdc_test


In [21]:
import pulsar

client = pulsar.Client(service_url,
                        authentication=pulsar.AuthenticationToken(token))
topic_partitions = client.get_topic_partitions(topic_url)
print(topic_partitions)

['persistent://cdc/astracdc/data-b04d69bb-6851-4bac-9703-8108dc560c41-cdc.cdc_test-partition-0', 'persistent://cdc/astracdc/data-b04d69bb-6851-4bac-9703-8108dc560c41-cdc.cdc_test-partition-1', 'persistent://cdc/astracdc/data-b04d69bb-6851-4bac-9703-8108dc560c41-cdc.cdc_test-partition-2']


In [19]:
subscription_name = input("subsuctiption name:")

subsuctiption name:cdcsub


In [26]:
!pip3 install 'pulsar-client[avro]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.4 MB/s eta 0:00:00


In [29]:
AvroSchema

pulsar.schema.schema_avro.AvroSchema

In [30]:
!pip3 install fastavro pytz

In [34]:
import pulsar
from pulsar.schema import *
import time

MODE_RUNNING = False

consumer = client.subscribe(topic_url, subscription_name)

try:
    waitingForMsg = True
    while waitingForMsg:
        try:
            msg = consumer.receive(2000)

            try:
                print("Received message '{}' id='{}'".format(msg.data(), msg.message_id()))
                # Acknowledging the message to remove from message backlog
                consumer.acknowledge(msg)
            except:
                consumer.negative_acknowledge(msg)

        except Exception as ex:
            print(ex)
            if MODE_RUNNING:
                print("Still waiting for a message...");
            else:
                print("Stop waiting for a message.");
                waitingForMsg = False
        time.sleep(1)

finally:
  consumer.close()

Received message 'b'\x02\x10good-by2'' id='(435404,2,2,-1)'
Pulsar error: TimeOut
Stop waiting for a message.


In [99]:
consumer = client.subscribe(topic_url, subscription_name,schema=JsonSchema(Message))

IncompatibleSchema: ignored

In [106]:
!pip3 install 'pulsar-client[all]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5894 sha256=1dc907753589a6f3b7ec2a24f2b548ae8aec6cad3c4e3ec8fededff23da06fc2
  Stored in directory: /root/.cache/pip/wheels/27/5f/ba/e972a56dcbf5de9f2b7d2b2a710113970bd173c4dcd3d2c902
Successfully built ratelimit


In [107]:
import pulsar
from pulsar.schema import *
consumer = client.subscribe(topic_url, subscription_name,schema=AvroSchema(Message()))

Exception: ignored

In [44]:
avro_schema_definition = """{
  "key": {
    "type": "record",
    "name": "cdc_test",
    "namespace": "_62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc",
    "doc": "Table 62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc.cdc_test",
    "fields": [
      {
        "name": "message_id",
        "type": "string"
      }
    ]
  },
  "value": {
    "type": "record",
    "name": "cdc_test",
    "namespace": "_62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc",
    "doc": "Table 62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc.cdc_test",
    "fields": [
      {
        "name": "message",
        "type": [
          "null",
          "string"
        ],
        "default": null
      }
    ]
  }
}"""

In [49]:
!pip install avro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for avro: filename=avro-1.11.3-py2.py3-none-any.whl size=123911 sha256=98fbf4a1da0526262c522e15557be45ce67a5fcbedf0013623e6ee03a5ef2b4a
  Stored in directory: /root/.cache/pip/wheels/1d/f6/41/0e0399396af07060e64d4e32c8bd259b48b98a4a114df31294
Successfully built avro


In [50]:
from avro import schema, io, datafile

In [52]:
avro_schema = schema.parse(avro_schema_definition)

SchemaParseException: ignored

In [71]:
avro_schema_definition = """{
  "key": {
    "type": "record",
    "name": "cdc_test",
    "namespace": "_62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc",
    "doc": "Table 62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc.cdc_test",
    "fields": [
      {
        "name": "message_id",
        "type": "string"
      }
    ]
  },
  "value": {
    "type": "record",
    "name": "cdc_test",
    "namespace": "_62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc",
    "doc": "Table 62303464363962622d363835312d346261632d393730332d383130386463353630633431_cdc.cdc_test",
    "fields": [
      {
        "name": "message",
        "type": [
          "string"
        ]
      }
    ]
  }
}"""

In [65]:
avro_schema_definition = """{
    "type": "record",
    "name": "cdc_test",
    "fields": [
        {"name": "message", "type": "string"},
        {"name": "message_id", "type": "string"},
    ]
}"""

In [73]:
avro_schema_definition = """{
    "type": "record",
    "name": "cdc_test",
    "fields": [
        {"name": "message", "type": "string"},
        {"name": "message_id", "type": "string"},
    ]
}"""

In [82]:
avro_schema = schema.parse("""
{"type": "record", "name": "cdc_test","fields": [
        {"name": "message", "type": "string"},
        {"name": "message_id", "type": "string"}

    ]}
""")#avro_schema_definition)

In [94]:
pulsar_schema = Schema.encode(obj=avro_schema)

TypeError: ignored

In [108]:
pulsar_schema = AvroSchema(avro_schema)

Exception: ignored

In [90]:
consumer = client.subscribe(
    topic_url,
    subscription_name,
    #consumer_configuration=consumer_configuration,
    schema=avro_schema#pulsar_schema
)

ValueError: ignored

In [97]:
pulsar_schema_avro = pulsar.schema.AvroSchema(avro_schema_definition)

Exception: ignored

In [96]:
!pip3 install 'pulsar-client[avro]'